In [ ]:
%maven org.dflib:dflib-jupyter:1.0.0-SNAPSHOT

In [ ]:
import org.dflib.csv.*;
import org.dflib.echarts.*;

import java.time.*;
import java.net.http.*;

In [ ]:
HttpRequest request = HttpRequest.newBuilder()
      .uri(URI.create("https://data.nasa.gov/api/views/gh4g-9sfh/rows.csv?accessType=DOWNLOAD"))
      .build();
String meteoritesCsv = HttpClient
    .newHttpClient()
    .send(request, HttpResponse.BodyHandlers.ofString()).body();

meteoritesCsv.substring(0, 500)    

In [ ]:

// the dataset source is https://catalog.data.gov/dataset/meteorite-landings
// annual stats on meteorite landings


int thisYear = Year.now().getValue();

DataFrame meteoriteStats = 

    // load data from CSV
    Csv.loader()
    .emptyStringIsNull()
    .load(new StringReader(meteoritesCsv))

    // cleanup data, and select a range
    .rows(and(
        $col("mass (g)").isNotNull(),
        $col("year").isNotNull(),
        $col("year").castAsInt().between(thisYear - 100, thisYear))).select()

    // calculate per-year stats
    .group("year")
    .agg(
        $col("year").castAsInt().first().as("year"), 
        count().as("count"),
        $str("mass (g)").castAsDecimal().scale(0).sum().as("mass, g")
    ).sort($col("year").asc());

meteoriteStats

In [ ]:
DataFrame forPlot = meteoriteStats.cols("year", "mass, kg").map(
        $int("year").mapVal(y -> LocalDate.of(y, 1, 1)),
        $decimal("mass, g").div(1000));

ECharts.chart()
    .xAxis("year", XAxis.ofTime())
    .yAxes(
        YAxis.ofValue().name("mass, kg"), 
        YAxis.ofValue().name("count").alignTicks(true))
    
    .series(SeriesOpts.bar(), "mass, kg")
    .series(SeriesOpts.line().yAxisIndex(1).smooth().showSymbol(false), "count")
    
    .plot(forPlot)